In [1]:
# pip install textblob vadersentiment nltk tensorflow

In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [120]:
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pickle as pkl
import time, nltk
import unicodedata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from sklearn.metrics import accuracy_score, precision_score, recall_score, make_scorer, log_loss
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, cross_validate
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from sklearn.svm import LinearSVC

import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# import time
# t0 = time.time()
# # your code here
# t1 = time.time()
# total_runtime = t1-t0 

# import pyspark as ps    # for the pyspark suite
# spark = (ps.sql.SparkSession
#          .builder
#          .master('local[4]')
#          .appName('lecture')
#          .getOrCreate()
#         )

# sc = spark.sparkContext

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
import os               # for environ variables in Part 3
from RottenorNot import extract_bow_from_raw_text
import json

df = pd.read_csv('rotten_tomatoes_reviews.csv')
df = df.drop_duplicates(subset=['Review', 'Freshness'])

In [4]:
#get X and y
y = df['Freshness'].to_numpy()
X = df["Review"]

           
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

In [18]:
# extracting bows
bows = list(map(lambda row: extract_bow_from_raw_text(row), X_train))

In [17]:
# y.shape, X.shape
# df['Freshness'].value_counts()

1    188127
0    151589
Name: Freshness, dtype: int64

In [19]:
pickle_out = open('myBows.pkl','wb')
pkl.dump(bows, pickle_out)
pickle_out.close()

pickle_in = open('myBows.pkl','rb')
bows = pkl.load(pickle_in)

In [5]:
# corpus = [row for row in X_train]

In [7]:
# from sklearn.feature_extraction.text import CountVectorizer

# tf = CountVectorizer(analyzer='word', max_features=10000, stop_words='english')

# document_tf_matrix = tf.fit_transform(corpus).todense()

In [8]:
# from math import log

# def idf(frequency_matrix):
#     df =  float(len(document_tf_matrix)) / sum(frequency_matrix > 0)
#     return [log(i) for i in df.getA()[0]]
# print(sorted(tf.vocabulary_))
# print(idf(document_tf_matrix))

In [88]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', max_features=100000, stop_words='english')
document_tfidf_matrix = tfidf.fit_transform(X_train)
# print(sorted(tfidf.vocabulary_))
# print(document_tfidf_matrix.todense())

In [130]:
model_log_regress = LogisticRegression(random_state=1)
random_forest = RandomForestClassifier()
gradient_boost = GradientBoostingClassifier()
tvec_score = cross_validate(model_log_regress, document_tfidf_matrix, y_train, cv=3, scoring=['accuracy', 'precision', 'recall', 'f1'])
# for key, val in tvec_score.items():
#     print(f'{key}: {val.mean()}')

In [136]:
forest_score = cross_validate(random_forest, document_tfidf_matrix, y_train, cv=3, scoring=['accuracy', 'precision', 'recall', 'f1'], verbose=4, n_jobs=-1)
for key, val in forest_score.items():
    print(f'{key}: {val.mean()}')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 18.5min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
gradient_score = cross_validate(gradient_boost, document_tfidf_matrix, y_train, cv=3, verbose=4, scoring=['accuracy', 'precision', 'recall', 'f1'], n_jobs=-1)
for key, val in gradient_score.items():
    print(f'{key}: {val.mean()}')

In [54]:
# model_grad_boost = GradientBoostingClassifier()
# model_rand_for = RandomForestClassifier()

KeyboardInterrupt: 

In [57]:
# # Create first pipeline for base without reducing features.

# pipe = Pipeline([('classifier' , LogisticRegression())])
# # pipe = Pipeline([('classifier', RandomForestClassifier())])

# # Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    ]

# # Create grid search object

# clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# # Fit on data

# best_clf = clf.fit(X_train, y_train)

# # print(f"log loss = {-clf.score(X_test, y_test)}")
# print(f"accuracy = {(clf.predict(X_test) == y_test).mean()}")
# print(clf.best_params_)
# print(clf.best_estimator_)

In [117]:
# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = Pipeline([
                ('tfidf', tfidf),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])




# params={'model__C':[.01,.05,.1,.5,1,5,10],
#            'model__penalty':['l1','l2']}


clf = GridSearchCV(NB_pipeline, cv=5, n_jobs=-1, param_grid=param_grid ,scoring=['f1', 'recall', 'precision'])
best_clf = clf.fit(X_train, y_train) 

NameError: name 'pipeline' is not defined

In [70]:
# grid_search = GridSearchCV(NB_pipeline, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)


# best_clf = grid_search.fit(X_train, y_train)

# # print(f"log loss = {-clf.score(X_test, y_test)}")
# print(f"accuracy = {(clf.predict(X_test) == y_test).mean()}")
# print(clf.best_params_)
# print(clf.best_estimator_)

In [51]:
# for param in grid_search.get_params().keys():
#     print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__verbose
estimator__tfidf
estimator__clf
estimator__tfidf__analyzer
estimator__tfidf__binary
estimator__tfidf__decode_error
estimator__tfidf__dtype
estimator__tfidf__encoding
estimator__tfidf__input
estimator__tfidf__lowercase
estimator__tfidf__max_df
estimator__tfidf__max_features
estimator__tfidf__min_df
estimator__tfidf__ngram_range
estimator__tfidf__norm
estimator__tfidf__preprocessor
estimator__tfidf__smooth_idf
estimator__tfidf__stop_words
estimator__tfidf__strip_accents
estimator__tfidf__sublinear_tf
estimator__tfidf__token_pattern
estimator__tfidf__tokenizer
estimator__tfidf__use_idf
estimator__tfidf__vocabulary
estimator__clf__estimator__alpha
estimator__clf__estimator__class_prior
estimator__clf__estimator__fit_prior
estimator__clf__estimator
estimator__clf__n_jobs
estimator
iid
n_jobs
param_grid
pre_dispatch
refit
return_train_score
scoring
verbose


In [96]:
# train the model using X_dtm & y
fitted_pipe = NB_pipeline.fit(X_train, y_train)
# compute the testing accuracy
prediction = NB_pipeline.predict(X_test)

prediction.shape

(67944,)

In [93]:
y_test.shape

# NB_pipeline.predict_proba(X_test)



(67944,)

In [97]:
### rmse of tfid and 
from sklearn.metrics import mean_squared_error  
from math import sqrt
mse = mean_squared_error(y_test, prediction)
rms = sqrt(mse)
rms

0.4704721543721392

In [115]:
model = GradientBoostingClassifier(learning_rate=0.1, max_depth=6,\
                                        max_features=0.3, min_samples_leaf=5,\
                                        n_estimators=500, random_state=1)model.fit(X_train, y_train)

In [116]:
grid_search()

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestClassifier()' (type <class 'sklearn.ensemble._forest.RandomForestClassifier'>) doesn't